# Проект по АвтОбрЕЯ

Ccылка на файл с требованиями: https://docs.google.com/document/d/1hHPc2y_fXNvlh623_5GTAMqcAtSucxNaC9ZRuwHi3Wk/edit#

### Над проектом работали:

*   Андрей Зырянов
*   Настя Каприелова
*   Настя Кромина
*   Петрова Даша

При выполнении задачи приветствуются эксперименты с теми методами, про которые рассказывали на занятиях: различные меры ассоциативной связи (коллокационные меры), разрешения семантической неоднозначности (AdaGram, семантическое расстояние по РуТез), определения семантической близости, тематическое моделирование, извлечение словаря тональной лексики…
Для морфологической разметки и выделения синтаксических связей можно использовать UDPipe. Кроме того, можно использовать различные дополнительные ресурсы: векторные модели / эмбеддинги (в том числе мультиязычные), любые другие корпуса (например, описания блюд для составления списка блюд)

**Формируем датасет для корпуса SentiRuEval_rest_train**

In [0]:
import re
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [0]:
corpus_train_file = open('SentiRuEval_rest_train.xml', 'r', encoding = 'utf-8', errors='ignore')
corpus_train = corpus_train_file.read()
corpus_train_file.close()

In [0]:
texts = re.findall(r'<text>(.*?)</text>', corpus_train)

In [0]:
food = re.findall(r'<food>(.*?)</food>', corpus_train)

In [0]:
service = re.findall(r'<service>(.*?)</service>', corpus_train)

In [0]:
d = {'texts': texts, 'food': food, 'service': service}
df = pd.DataFrame(data=d)

In [10]:
df.head()

,texts,food,service
0,И пускай на меня не обижается наш прославленны...,8,8
1,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,...",9,10
2,"Советую вам уволить Вашего метродотеля Елену, ...",9,1
3,отличный средне вековый интеръер. Приятное обс...,8,9
4,Ужинали в ресторане Баден-Баден 6 марта . Импо...,10,8


In [11]:
!pip install pymorphy2

In [0]:
#preprocessing texts in the df
import pymorphy2

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

p = re.compile(r'[\W^ ]')
morph = pymorphy2.MorphAnalyzer()

In [0]:
updated_texts = []
for row in df['texts'][:5000]:
  clean = p.sub(' ', row)
  words = clean.split()
  new_text = ' '.join(words)
  updated_texts.append(new_text)

In [0]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [morph.parse(t)[0].normal_form for t in tokens]
    return lemmas


def get_ngrams(tokens, n, patterns=None):
    ngrams = []
    for i in range(len(tokens) - (n - 1)):
        ngram = tokens[i:i+n]
        tags = [morph.parse(t)[0].tag.POS for t in ngram]
        if patterns is not None:
            if tags in patterns:
                ngrams.append(ngram)
        else:
            ngrams.append(ngram)
    return ngrams

In [0]:
import time

Создадим списки с биграммами и триграммами для проверки сочетаемости слов

In [16]:
start_time = time.time()
all_bigrams_adj = []
for text in updated_texts: 
  bi_grams = get_ngrams(normalize(text), n=2, patterns=[['ADJF', 'NOUN']])
  all_bigrams_adj.append(bi_grams)
print("--- %s seconds ---" % (time.time() - start_time))

--- 476.0370469093323 seconds ---


In [0]:
start_time = time.time()
all_trigrams_adv_adj = []
for text in updated_texts: 
  tr_grams_adv_adj = get_ngrams(normalize(text), n=3, patterns=[['ADVB', 'ADJF', 'NOUN']])
  all_trigrams_adv_adj.append(tr_grams_adv_adj)
print("--- %s seconds ---" % (time.time() - start_time))

In [0]:
start_time = time.time()
all_trigrams_adv_verb = []
for text in updated_texts: 
  tr_grams_adv_verb = get_ngrams(normalize(text), n=3, patterns=[['NOUN', 'ADVB', 'INFN']])
  all_trigrams_adv_verb.append(tr_grams_adv_verb)
print("--- %s seconds ---" % (time.time() - start_time))

In [0]:
#достаем из данных нам списков слова, которые можно отнести к Food 0, Food 1, Service 0 или Service 1

In [0]:
Food_file = open('Food_words.txt', 'r', encoding = 'utf-8')
Food = Food_file.read()
Food_file.close()

In [0]:
Food_split = Food.split('\n')

In [0]:
Food0 = []
Food1 = []
for i in Food_split:
  r = i.split('        ')
  if r[2] == '1':
    Food1.append(r[1])
  if r[2] == '0':
    Food0.append(r[1])

In [0]:
Service_file = open('Service_words.txt', 'r', encoding = 'utf-8')
Service = Service_file.read()
Service_file.close()

In [0]:
Service_split = Service.split('\n')

In [0]:
Service0 = []
Service1 = []
for i in Service_split:
  r1 = i.split('\t')
  if r1[2] == '1':
    Service1.append(r1[1])
  if r1[2] == '0':
    Service0.append(r1[1])

Загрузим модель, чтобы использовать её эмбеддинги.

In [0]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

In [0]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import re
def tokenize(line): #функция возвращает список токенов данного предложения
  ws = []
  words = line.split()
  for w in words:
    w = re.sub('[.,-;:?!@#$%^&()_+=—–"…}{/\|«»>]', '', w).lower()
    if w != "":
      p = morph.parse(w)[0]
      ws.append(p.normal_form)
  return ws

In [0]:
from numpy import dot
from numpy.linalg import norm

def cossim(v1,v2): # функция подсчета косинусного расстояния
  return dot(v1, v2)/(norm(v1)*norm(v2))

In [26]:
model_file = 'model.model'
model = KeyedVectors.load(model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [27]:
query = 'Картошка была прекрасна.'
query_tok = tokenize(query)
query_tok

['картошка', 'быть', 'прекрасный']

In [28]:
query_vec = []
for i in query_tok:
  res = i in model.wv
  print(i, ': ', res)
  print(model[i][:5])
  query_vec.append(model[i])

картошка :  True
[-0.01691983 -0.5142729   0.31042105  0.5475326  -0.28163046]
быть :  True
[ 0.1256133  -0.54293644  0.24227035  0.04432327  0.1083925 ]
прекрасный :  True
[ 0.12641275 -0.31853437  0.04751264 -0.06571354  0.1707755 ]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Создадим дополнительные сиды, а также сделаем массивы их векторов.

In [0]:
seed_food = ['мясо','рыба','лук','гриб', 'помидор']
food_vec = []
for i in seed_food:
  food_vec.append(model[i])

In [0]:
seed_serv = ['официант','обслуживание','сервис','администратор', 'сотрудник']
serv_vec = []
for i in seed_serv:
  serv_vec.append(model[i])

In [31]:
for i in range(len(query_vec)):
  total = 0
  v1 = query_vec[i]
  for j in range(len(food_vec)):
    v2 = food_vec[j]
    sim = cossim(v1,v2)
    total += sim
  print(f'Близость слова "{query_tok[i]}" к food_seed = {total}')

Близость слова "картошка" к food_seed = 2.6930912733078003
Близость слова "быть" к food_seed = 1.044472575187683
Близость слова "прекрасный" к food_seed = 0.7706841230392456


In [0]:
seed_pos = []
for i in Food1:
  seed_pos.append(i)
for i in Service1:
  seed_pos.append(i)

In [33]:
len(seed_pos)

36

In [0]:
seed_pos = ['отличный','вкусный','сочный','удобный']

In [0]:
pos_vec = []
for i in seed_pos:
  pos_vec.append(model[i])

In [0]:
seed_neg = []
for i in Food0:
  seed_neg.append(i)
for i in Service0:
  seed_neg.append(i)

In [37]:
len(seed_neg)

4

In [0]:
seed_neg = ['ужасный','кошмарный','отвратительный','злой']

In [0]:
neg_vec = []
for i in seed_neg:
  neg_vec.append(model[i])

In [0]:
#кандидаты по близости
food_cand = []
serv_cand = []
pos_cand = []
neg_cand = []

#их вектора
food_vec_cand = []
serv_vec_cand = []
pos_vec_cand = []
neg_vec_cand = []

Функция detect находит в предложениии кандидатов для пополнения списков seed и выдает результат в соответствующем формате

Для задачи нахождения оценочных слов с каким-то аспектом в предложении, формат вывода отличается, поэтому для его переформатирования используется функция  result_transform

В функции detect мы для каждого слова из предложения смотрим его косинуную близость со словами из сидов, если число близостей, которые > 0.3, превышает 75% от сида, добавляем слово в кандидаты для пополнения сида.

Если в одном предложении встречается слово с аспектом и оценочное слово, пара становится кандидатом для списка оценочных слов.

Все кандидаты далее будут проверяться на основе сравнения с коллокациями.

In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('russian'))

In [0]:
candidates = []

In [0]:
def detect(sent): # для одного предложения
  result = []
  old_food_len = len(food_cand)
  old_serv_len = len(serv_cand)
  old_pos_len = len(pos_cand)
  old_neg_len = len(neg_cand)

  words = tokenize(sent)
  food = 'no food'
  service = 'no service'

  for i in range(len(words)):
    if words[i] not in stopwords:
      sim_pass = 0
      for j in food_vec:  #цикл для еды
        sim = cossim(model[words[i]], j)
        if sim > 0.3:
          sim_pass += 1
      if sim_pass > 3:
        food = 'detected'
        if words[i] not in food_cand: 
          food_cand.append(words[i])
          food_vec_cand.append(model[words[i]])
      new_food_len = len(food_cand)

      sim_pass_serv = 0
      for s in serv_vec: #цикл для сервиса
        sim = cossim(model[words[i]], s)
        if sim > 0.3:
          sim_pass_serv += 1
      if sim_pass_serv > 3:
        service = 'detected'
        if words[i] not in serv_cand: 
          serv_cand.append(words[i])
          serv_vec_cand.append(model[words[i]])
      new_serv_len = len(serv_cand)

      sim_pass_pos = 0  
      for p in pos_vec: #цикл для положительных слов
        sim = cossim(model[words[i]], p)
        if sim > 0.3:
          sim_pass_pos += 1
      if sim_pass_pos > 3 and words[i] not in pos_cand: 
        pos_cand.append(words[i])
        pos_vec_cand.append(model[words[i]])
      new_pos_len = len(pos_cand)

      sim_pass_neg = 0
      for n in neg_vec: #цикл для негативных слов
        sim = cossim(model[words[i]], n)
        if sim > 0.3:
          sim_pass_neg += 1
      if sim_pass_neg > 3 and words[i] not in neg_cand: 
        neg_cand.append(words[i])
        neg_vec_cand.append(model[words[i]])
      new_neg_len = len(neg_cand)

  if food == 'detected' and service == 'no service': # если есть еда, но нет сервиса
    if old_pos_len != new_pos_len: # если новое положительное слово
      result.append(f'Food\t{words[i]}\t1')
      candidates.append(f'Food\t{words[i]}\t1')
    elif old_neg_len != new_neg_len: # если новое отрицательное слово
      result.append(f'Food\t{words[i]}\t0')
      candidates.append(f'Food\t{words[i]}\t0')

  if service == 'detected' and food == 'no food': # если есть сервис, но нет еды
    if old_pos_len != new_pos_len: # если новое положительное слово
      result.append(f'Service\t{words[i]}\t1')
      candidates.append(f'Service\t{words[i]}\t1')
    elif old_neg_len != new_neg_len: # если новое отрицательное слово
      result.append(f'Service\t{words[i]}\t0')
      candidates.append(f'Service\t{words[i]}\t0')
      
  return result

In [0]:
sent_tt = 'Официантка была грубой.'

In [45]:
detect(sent_tt)

['Service\tгрубый\t0']

In [46]:
print(food_cand)
print(serv_cand)
print(pos_cand)
print(neg_cand)

[]
['официантка']
[]
['грубый']


In [0]:
def text_test(text): #detect для текста
  sents = []
  pattern = re.compile(r'([А-ЯA-Z]((т.п.|т.д.|пр.|г.)|[^?!.\(]|\([^\)]*\))*[.?!]*)')
  for i,sent in enumerate(pattern.findall(text)):
    sents.append(sent[0])
  res = [] # результаты по всем предложениям текста
  for i in sents:
    result = detect(i)
    if result != []:
      #new_form = result_transform(result):
      #res.append(new_form)
      res.append(result)
  
  return res

In [50]:
test_text = 'Еда была очень вкусной. Официантка была злой и грубой!'
test_res = text_test(text)
test_res

[['Food\tвкусный\t1'], ['Service\tгрубый\t0']]

После обработки 1000 текстов, мы нашли следующих кандидатов для списков слов.

In [55]:
print(food_cand)
print(serv_cand)
print(pos_cand)
print(neg_cand)

['еда', 'вкусный', 'поесть', 'вкусно', 'мясо', 'кондитерский', 'мясной', 'запеканка', 'паста', 'говядина', 'пицца', 'тесто', 'креветка', 'макрель', 'пиво', 'поедать', 'фрукт', 'сервировка', 'шампанский', 'рыба', 'кролик', 'белуга', 'сакура', 'каштан', 'десерт', 'блюдо', 'пиццерия', 'харчо', 'невкусно', 'полуфабрикат', 'закуска', 'казалось', 'жареный', 'молодая', 'осьминог', 'шоколадный', 'съесть', 'сало', 'клешня', 'пельмень', 'колбаска', 'баранина', 'баклажан', 'торт', 'мороженое']
['официантка', 'обслуживание', 'ресторан', 'заведение', 'клиент', 'кондитерский', 'комендантский', 'администратор', 'сотрудник', 'организация', 'банкетный', 'заказ', 'парковский', 'ресторанчик', 'кафешка', 'посетитель', 'персонал', 'оператор', 'автосалон', 'интерьер', 'сервис', 'ресторанный', 'официант', 'работник', 'работодатель', 'зарплата', 'обслуживаться', 'корпоративный', 'пиццерия', 'турист']
['вкусный', 'красивый', 'оригинальный', 'шикарный', 'приятный', 'пристойный', 'симпатичный', 'привлекательный'

In [0]:
ress = []
for i in texts[:2000]:
  try:
    ress.append(text_test(i))
  except AttributeError:
    continue

После 2000 пополнили еще.

In [62]:
print(food_cand)
print(serv_cand)
print(pos_cand)
print(neg_cand)

['еда', 'вкусный', 'поесть', 'вкусно', 'мясо', 'кондитерский', 'мясной', 'запеканка', 'паста', 'говядина', 'пицца', 'тесто', 'креветка', 'макрель', 'пиво', 'поедать', 'фрукт', 'сервировка', 'шампанский', 'рыба', 'кролик', 'белуга', 'сакура', 'каштан', 'десерт', 'блюдо', 'пиццерия', 'харчо', 'невкусно', 'полуфабрикат', 'закуска', 'казалось', 'жареный', 'молодая', 'осьминог', 'шоколадный', 'съесть', 'сало', 'клешня', 'пельмень', 'колбаска', 'баранина', 'баклажан', 'торт', 'мороженое', 'блин', 'салат', 'поварить', 'жаркое', 'птица', 'капуста', 'морс', 'столовый', 'клюква', 'мамалыга', 'пирожное', 'пирог', 'рыбный', 'мидия', 'корзиночка', 'малина', 'сливка', 'водка', 'кишмиш', 'шашлык', 'отведать', 'гюльчатай', 'вино', 'продукт', 'ложка', 'яйцо', 'овощной', 'сардина', 'стейк', 'телятина', 'имбирь', 'фасоль', 'кабачок', 'брынза', 'пётр', 'артишок', 'сыр']
['официантка', 'обслуживание', 'ресторан', 'заведение', 'клиент', 'кондитерский', 'комендантский', 'администратор', 'сотрудник', 'организ

Видно, что цикл для пар очень строгий и они почти не находятся. Поэтому мы решили при помощи исследования коллокаций и расширенных сидов определять аспекты для выделенных кандидатов.

In [61]:
candidates

['Service\tолимпий\t0', 'Service\tпаб\t1']

Ниже функция, которая использует распарсенные udpipe данные, чтобы находить токен нужного слова

In [0]:
import re

with open('sentences_parsed', 'r', encoding = 'utf-8') as f:
    parsed = f.read()
    texts = parsed.split('SpacesAfter=\\r\\n')
    reviews = []
    for text in texts:
        sentences_raw = text.split('# sent_id')
        sentences = []
        for sent_raw in sentences_raw[1:]:
            words = sent_raw.split('\n')[2:]
            sentences.append(words)
        reviews.append(sentences)

def searchsent(i_rev, i_sent, word):
    i_out = []
    lemma = normalize(word)[0]
    for line in reviews[i_rev][i_sent]:
        if re.findall(lemma, line) != []:
            i_out.append(line.split('\t')[0])
    return i_out

In [51]:

#def result_transform(result): #функция, которая меняет формат выходных данных функции detect в соответствии с заданием
  


['Food\tвкусный\t1']
['Service\tгрубый\t0']
